<h1>db-index, new way to measure dataset bias</h1>

In [2]:
from datasets import load_dataset
import spacy as sp
import random
import en_core_web_lg
import pandas as pd

In [3]:
nlp = en_core_web_lg.load()

In [4]:
dataset = load_dataset("ccdv/govreport-summarization")
dataset

No config specified, defaulting to: govreport-summarization/document


Generating train split: 0 examples [00:00, ? examples/s]

DatasetGenerationError: An error occurred while generating the dataset

In [4]:
dataset.set_format(type='pandas')
targetdset = dataset['train'][1:]
targetdset.drop(['summary'], axis=1)

,report
0,Most income derived from private sector busine...
1,There are some similarities in how Medicare pa...
2,IRS’s mission is to provide America’s taxpayer...
3,"In 1991, we reported that, historically, INS l..."
4,While TCE and perchlorate are both DOD-classif...
...,...
17511,Some Members of Congress have expressed intere...
17512,"Since November 1986, the Commemorative Works A..."
17513,"On May 27, 2015, the Army Corps of Engineers (..."
17514,Section 1512 applies to the obstruction of fed...


Dataset sanitized. Let's get a sample dataset of offensive language

In [5]:
offensivedataset = load_dataset("henryscheible/implicit_bias")

OSError: [Errno 28] No space left on device: '/home/devam/.cache/huggingface/datasets/henryscheible___parquet'

In [6]:
offensivedataset.set_format(type='pandas')
compdataset = offensivedataset['train'][1:]
compdataset.drop(['category', 'label'], axis = 1)

NameError: name 'offensivedataset' is not defined

The history saving thread hit an unexpected error (OperationalError('unable to open database file')).History will not be written to the database.


In [7]:
def dbindex(target, comparison):
    #Pick a random entry from the comparison dataset.
    index = random.randrange(0, len(comparison), 1)

    #Find that entry.
    comparisonEntry = comparison['sentence'][index]

    #Vectoritze entry.
    vectorizedComparisonEntry = nlp(comparisonEntry)

    #initialitize total cosine similarity
    tcs = 0

    #Go through each target entry
    for entry in target.report:
        
        #Find similariity between each target and the comparison entry
        vecEntry = nlp(entry)
        tcs += vecEntry.similarity(vectorizedComparisonEntry)

    #Raise to inverse power of size
    dbi = tcs**(1/len(target))
    return dbi

In [8]:
target = pd.DataFrame(["grey", "grey"])
target.columns = ['report']
comparison = pd.DataFrame(['white', "red", "orange"])
comparison.columns = ['sentence']

In [9]:
dbindex(targetdset, compdataset)

NameError: name 'targetdset' is not defined